In [ ]:
#| default_exp sandbox

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| hide
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv("../.env")

In [ ]:
os.environ['HTTP_PROXY'] = "http://127.0.0.1:20171"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:20171"
os.environ['http_proxy'] = "http://127.0.0.1:20171"
os.environ['https_proxy'] = "http://127.0.0.1:20171"

In [ ]:
os.getenv('GC_QUOTE_API_CREDENTIALS')
os.getenv('HTTP_PROXY')
os.getenv('HTTPS_PROXY')

In [ ]:
from dotenv import dotenv_values
config = dotenv_values("../.env")
key = config.get('GC_QUOTE_API_CREDENTIALS')

In [ ]:
from google.cloud import bigquery
import pandas_gbq
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../" + key)

In [ ]:
client = bigquery.Client(credentials=credentials)

QUERY = """
		select *
		from `quote-api-365206.report.ride_dispatch_v` r
		where dispatch_amount_net_usd is not null
        limit 200
		"""

In [ ]:
QUERY_DISINCT = """
		select count (*)
		from `quote-api-365206.report.ride_dispatch_v` r
		where trip_type is null
        limit 200
		"""

In [ ]:
!echo $HTTP_PROXY

In [ ]:
#query_job = client.query(QUERY)

In [ ]:

df = pandas_gbq.read_gbq(QUERY,credentials=credentials)

In [ ]:

df_trip_type = pandas_gbq.read_gbq(QUERY_DISINCT,credentials=credentials)

In [ ]:

df.head()

In [ ]:
QUERY_ROUTE = """
        SELECT r.*, p1.lat from_lat, p1.lng from_lng, p2.lat to_lat, p2.lng to_lng
        FROM `report.ride_dispatch_v` r
        left JOIN `report.dim_place` p1
        ON r.pickup_place_id=p1.id
        INNER JOIN `report.dim_place` p2
        ON r.pickup_place_id=p2.id
        ORDER BY ride_id DESC
        limit 2;
		"""


In [ ]:
df_route = pandas_gbq.read_gbq(QUERY_ROUTE,credentials=credentials)

In [ ]:
df_route.head()

In [ ]:
df_route.dtypes